In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Image Emotion Model Training\n",
    "\n",
    "This notebook trains a CNN-based model for facial emotion recognition using MobileNetV2."
   ]\n  }\n ],\n \"metadata\": {\n  \"kernelspec\": {\n   \"display_name\": \"Python 3\",\n   \"language\": \"python\",\n   \"name\": \"python3\"\n  },\n  \"language_info\": {\n   \"codemirror_mode\": {\n    \"name\": \"ipython\",\n    \"version\": 3\n   },\n   \"file_extension\": \".py\",\n   \"mimetype\": \"text/x-python\",\n   \"name\": \"python\",\n   \"nbconvert_exporter\": \"python\",\n   \"pygments_lexer\": \"ipython3\",\n   \"version\": \"3.8.5\"\n  }\n },\n \"nbformat\": 4,\n \"nbformat_minor\": 4\n}
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import sys\n",
    "import os\n",
    "sys.path.append('../src')\n",
    "\n",
    "import pandas as pd\n",
    "import numpy as np\n",
    "import torch\n",
    "import torch.nn as nn\n",
    "import torchvision.transforms as transforms\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "import cv2\n",
    "from PIL import Image\n",
    "from sklearn.metrics import classification_report, confusion_matrix\n",
    "import warnings\n",
    "warnings.filterwarnings('ignore')\n",
    "\n",
    "from data_loader import DataLoader\n",
    "from model_trainer import ModelTrainer, ImageEmotionModel\n",
    "from utils import EmotionUtils, ModelUtils, ImageUtils\n",
    "\n",
    "# Set random seeds for reproducibility\n",
    "torch.manual_seed(42)\n",
    "np.random.seed(42)\n",
    "\n",
    "# Check GPU availability\n",
    "device = ModelUtils.get_device()\n",
    "print(f\"Using device: {device}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Load and Prepare Image Data"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Initialize data loader\n",
    "data_loader = DataLoader(data_dir='../data/')\n",
    "\n",
    "# Load processed image data\n",
    "processed_data_path = '../data/processed/image_emotions.csv'\n",
    "\n",
    "if os.path.exists(processed_data_path):\n",
    "    df = pd.read_csv(processed_data_path)\n",
    "    image_paths = df['image_path'].tolist()\n",
    "    labels = df['emotion'].tolist()\n",
    "    print(f\"Loaded {len(image_paths)} image samples from processed data\")\n",
    "else:\n",
    "    print(\"Processed data not found. Loading raw data...\")\n",
    "    # Try to load from FER-2013\n",
    "    fer_data_path = '../data/fer2013/'\n",
    "    \n",
    "    if os.path.exists(fer_data_path):\n",
    "        image_paths, labels = data_loader.load_fer2013_data(fer_data_path)\n",
    "    else:\n",
    "        print(\"FER-2013 dataset not found. Creating synthetic data for demonstration...\")\n",
    "        # Create synthetic image data for demonstration\n",
    "        # In real scenario, you would have actual image files\n",
    "        \n",
    "        # Create sample images directory\n",
    "        os.makedirs('../data/sample_images', exist_ok=True)\n",
    "        \n",
    "        image_paths = []\n",
    "        labels = []\n",
    "        \n",
    "        # Generate sample grayscale images for each emotion\n",
    "        for emotion in EmotionUtils.EMOTION_LABELS:\n",
    "            for i in range(20):  # 20 samples per emotion\n",
    "                # Create a random 48x48 grayscale image\n",
    "                img = np.random.randint(0, 256, (48, 48), dtype=np.uint8)\n",
    "                \n",
    "                # Add some pattern based on emotion (just for demo)\n",
    "                if emotion == 'happy':\n",
    "                    img = np.clip(img + 50, 0, 255)  # Brighter\n",
    "                elif emotion == 'sad':\n",
    "                    img = np.clip(img - 50, 0, 255)  # Darker\n",
    "                \n",
    "                img_path = f'../data/sample_images/{emotion}_{i:03d}.png'\n",
    "                cv2.imwrite(img_path, img)\n",
    "                \n",
    "                image_paths.append(img_path)\n",
    "                labels.append(emotion)\n",
    "        \n",
    "        print(f\"Created {len(image_paths)} synthetic image samples for training\")\n",
    "\n",
    "print(f\"\\nUnique emotions: {set(labels)}\")\n",
    "print(f\"Label distribution:\")\n",
    "from collections import Counter\n",
    "label_counts = Counter(labels)\n",
    "for emotion, count in label_counts.items():\n",
    "    print(f\"  {emotion}: {count}\")\n",
    "\n",
    "# Filter out non-existent image paths\n",
    "valid_paths = []\n",
    "valid_labels = []\n",
    "for path, label in zip(image_paths, labels):\n",
    "    if os.path.exists(path):\n",
    "        valid_paths.append(path)\n",
    "        valid_labels.append(label)\n",
    "\n",
    "image_paths = valid_paths\n",
    "labels = valid_labels\n",
    "print(f\"\\nValid image files: {len(image_paths)}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Visualize sample images\n",
    "print(\"Sample Images:\")\n",
    "\n",
    "# Select a few images from each emotion\n",
    "sample_images = {}\n",
    "for emotion in EmotionUtils.EMOTION_LABELS:\n",
    "    emotion_paths = [path for path, label in zip(image_paths, labels) if label == emotion]\n",
    "    if emotion_paths:\n",
    "        sample_images[emotion] = emotion_paths[0]  # Take first image of each emotion\n",
    "\n",
    "if sample_images:\n",
    "    fig, axes = plt.subplots(2, 4, figsize=(12, 6))\n",
    "    axes = axes.flatten()\n",
    "    \n",
    "    for i, (emotion, img_path) in enumerate(sample_images.items()):\n",
    "        if i < 8:  # Only show first 8\n",
    "            try:\n",
    "                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)\n",
    "                if img is not None:\n",
    "                    axes[i].imshow(img, cmap='gray')\n",
    "                    axes[i].set_title(f'{emotion.capitalize()}')\n",
    "                    axes[i].axis('off')\n",
    "                else:\n",
    "                    axes[i].text(0.5, 0.5, f'{emotion}\\n(No image)', \n",
    "                               ha='center', va='center', transform=axes[i].transAxes)\n",
    "                    axes[i].axis('off')\n",
    "            except Exception as e:\n",
    "                axes[i].text(0.5, 0.5, f'{emotion}\\n(Error loading)', \n",
    "                           ha='center', va='center', transform=axes[i].transAxes)\n",
    "                axes[i].axis('off')\n",
    "    \n",
    "    plt.suptitle('Sample Images by Emotion')\n",
    "    plt.tight_layout()\n",
    "    plt.show()\nelse:\n",
    "    print(\"No sample images available to display.\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create data loaders\n",
    "print(\"Creating data loaders...\")\n",
    "\n",
    "# Configuration\n",
    "BATCH_SIZE = 32\n",
    "TEST_SIZE = 0.2\n",
    "IMAGE_SIZE = (48, 48)\n",
    "\n",
    "if len(image_paths) > 0:\n",
    "    train_loader, val_loader, label_encoder = data_loader.create_image_dataloaders(\n",
    "        image_paths=image_paths,\n",
    "        labels=labels,\n",
    "        batch_size=BATCH_SIZE,\n",
    "        test_size=TEST_SIZE\n",
    "    )\n",
    "    \n",
    "    print(f\"Training batches: {len(train_loader)}\")\n",
    "    print(f\"Validation batches: {len(val_loader)}\")\n",
    "    print(f\"Classes: {label_encoder.classes_}\")\nelse:\n",
    "    print(\"❌ No valid image data available for training\")\n",
    "    train_loader = None\n",
    "    val_loader = None\n",
    "    label_encoder = None"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Initialize Model and Trainer"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Initialize model trainer\n",
    "model_trainer = ModelTrainer(device=device)\n",
    "\n",
    "# Training configuration\n",
    "training_config = {\n",
    "    \"model_architecture\": \"MobileNetV2\",\n",
    "    \"num_epochs\": 10,  # Reduced for demo\n",
    "    \"learning_rate\": 0.001,\n",
    "    \"batch_size\": BATCH_SIZE,\n",
    "    \"image_size\": IMAGE_SIZE,\n",
    "    \"num_emotions\": len(EmotionUtils.EMOTION_LABELS),\n",
    "    \"device\": str(device),\n",
    "    \"pretrained\": True\n",
    "}\n",
    "\n",
    "print(\"Training Configuration:\")\n",
    "for key, value in training_config.items():\n",
    "    print(f\"  {key}: {value}\")\n",
    "\n",
    "# Save training config\n",
    "os.makedirs('../models/image_emotion_model', exist_ok=True)\n",
    "model_trainer.save_training_config(training_config, '../models/image_emotion_model/config.json')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Train the Model"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Train the image emotion model\n",
    "if train_loader is not None and val_loader is not None:\n",
    "    print(\"Starting model training...\")\n",
    "    \n",
    "    model_save_path = '../models/image_emotion_model/model.pth'\n",
    "    \n",
    "    history = model_trainer.train_image_model(\n",
    "        train_loader=train_loader,\n",
    "        val_loader=val_loader,\n",
    "        model_save_path=model_save_path,\n",
    "        num_epochs=training_config[\"num_epochs\"],\n",
    "        learning_rate=training_config[\"learning_rate\"]\n",
    "    )\n",
    "    \n",
    "    print(\"\\n✅ Training completed!\")\nelse:\n",
    "    print(\"❌ Cannot start training - no valid data available\")\n",
    "    # Create a dummy history for demonstration\n",
    "    history = {\n",
    "        'train_loss': [0.8, 0.6, 0.4, 0.3, 0.2],\n",
    "        'val_loss': [0.9, 0.7, 0.5, 0.4, 0.35],\n",
    "        'train_acc': [60, 70, 80, 85, 90],\n",
    "        'val_acc': [55, 65, 75, 80, 82]\n",
    "    }"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Visualize Training Results"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Plot training history\n",
    "if 'history' in locals():\n",
    "    model_trainer.plot_training_history(\n",
    "        history, \n",
    "        save_path='../models/image_emotion_model/training_history.png'\n",
    "    )\nelse:\n",
    "    print(\"No training history available to plot\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Evaluate the Model"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Load and evaluate the trained model\n",
    "model_save_path = '../models/image_emotion_model/model.pth'\n",
    "\n",
    "if os.path.exists(model_save_path) and val_loader is not None:\n",
    "    # Load the best trained model\n",
    "    best_model = model_trainer.load_trained_model(model_save_path, \"image\")\n",
    "    \n",
    "    if best_model is not None:\n",
    "        print(\"Model loaded successfully!\")\n",
    "        \n",
    "        # Evaluate on validation set\n",
    "        print(\"\\nEvaluating model...\")\n",
    "        evaluation_results = model_trainer.evaluate_model(\n",
    "            model=best_model,\n",
    "            test_loader=val_loader,\n",
    "            model_type=\"image\"\n",
    "        )\n",
    "        \n",
    "        print(f\"\\nValidation Accuracy: {evaluation_results['accuracy']:.4f}\")\n",
    "        \n",
    "        # Print classification report\n",
    "        print(\"\\nClassification Report:\")\n",
    "        report = evaluation_results['classification_report']\n",
    "        for emotion in EmotionUtils.EMOTION_LABELS:\n",
    "            if emotion in report:\n",
    "                metrics = report[emotion]\n",
    "                print(f\"  {emotion}:\")\n",
    "                print(f\"    Precision: {metrics['precision']:.3f}\")\n",
    "                print(f\"    Recall: {metrics['recall']:.3f}\")\n",
    "                print(f\"    F1-score: {metrics['f1-score']:.3f}\")\n",
    "        \n",
    "        # Plot confusion matrix\n",
    "        model_trainer.plot_confusion_matrix(\n",
    "            evaluation_results['confusion_matrix'],\n",
    "            save_path='../models/image_emotion_model/confusion_matrix.png'\n",
    "        )\n",
    "    else:\n",
    "        print(\"❌ Failed to load trained model\")\nelse:\n",
    "    print(\"❌ No trained model available or no validation data\")\n",
    "    best_model = None"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. Test Model with Sample Images"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Test the model with sample images\n",
    "if best_model is not None and len(image_paths) > 0:\n",
    "    print(\"Testing model with sample images:\\n\")\n",
    "    \n",
    "    # Select a few test images\n",
    "    test_indices = np.random.choice(len(image_paths), min(5, len(image_paths)), replace=False)\n",
    "    \n",
    "    fig, axes = plt.subplots(1, len(test_indices), figsize=(15, 3))\n",
    "    if len(test_indices) == 1:\n",
    "        axes = [axes]\n",
    "    \n",
    "    for i, idx in enumerate(test_indices):\n",
    "        img_path = image_paths[idx]\n",
    "        true_label = labels[idx]\n",
    "        \n",
    "        # Load and preprocess image\n",
    "        try:\n",
    "            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)\n",
    "            if img is not None:\n",
    "                # Predict emotions\n",
    "                emotion_scores = model_trainer.predict_single_image(\n",
    "                    model=best_model,\n",
    "                    image=img\n",
    "                )\n",
    "                \n",
    "                # Get predicted emotion\n",
    "                predicted_emotion = max(emotion_scores.items(), key=lambda x: x[1])[0]\n",
    "                confidence = emotion_scores[predicted_emotion]\n",
    "                \n",
    "                # Display image\n",
    "                axes[i].imshow(img, cmap='gray')\n",
    "                axes[i].set_title(f'True: {true_label}\\nPred: {predicted_emotion}\\n({confidence:.2f})')\n",
    "                axes[i].axis('off')\n",
    "                \n",
    "                print(f\"Image {i+1}: {img_path}\")\n",
    "                print(f\"  True emotion: {true_label}\")\n",
    "                print(f\"  Predicted emotion: {predicted_emotion} (confidence: {confidence:.3f})\")\n",
    "                \n",
    "                # Show top 3 predictions\n",
    "                sorted_emotions = sorted(emotion_scores.items(), key=lambda x: x[1], reverse=True)[:3]\n",
    "                print(\"  Top predictions:\")\n",
    "                for emotion, score in sorted_emotions:\n",
    "                    emoji = EmotionUtils.get_emotion_emoji(emotion)\n",
    "                    print(f\"    {emoji} {emotion}: {score:.3f}\")\n",
    "                print()\n",
    "            else:\n",
    "                axes[i].text(0.5, 0.5, 'Failed to\\nload image', \n",
    "                           ha='center', va='center', transform=axes[i].transAxes)\n",
    "                axes[i].axis('off')\n",
    "        except Exception as e:\n",
    "            print(f\"Error processing image {img_path}: {e}\")\n",
    "            axes[i].text(0.5, 0.5, f'Error:\\n{str(e)[:20]}', \n",
    "                       ha='center', va='center', transform=axes[i].transAxes)\n",
    "            axes[i].axis('off')\n",
    "    \n",
    "    plt.tight_layout()\n",
    "    plt.show()\nelse:\n",
    "    print(\"❌ Model not available for testing or no images available\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 7. Model Performance Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Analyze model performance by emotion\n",
    "if 'evaluation_results' in locals():\n",
    "    print(\"Model Performance Analysis:\\n\")\n",
    "    \n",
    "    # Extract metrics for each emotion\n",
    "    emotions_data = []\n",
    "    report = evaluation_results['classification_report']\n",
    "    \n",
    "    for emotion in EmotionUtils.EMOTION_LABELS:\n",
    "        if emotion in report:\n",
    "            emotions_data.append({\n",
    "                'emotion': emotion,\n",
    "                'precision': report[emotion]['precision'],\n",
    "                'recall': report[emotion]['recall'],\n",
    "                'f1_score': report[emotion]['f1-score'],\n",
    "                'support': report[emotion]['support']\n",
    "            })\n",
    "    \n",
    "    # Create DataFrame for easier analysis\n",
    "    if emotions_data:\n",
    "        performance_df = pd.DataFrame(emotions_data)\n",
    "        \n",
    "        print(\"Per-Emotion Performance:\")\n",
    "        print(performance_df.round(3))\n",
    "        \n",
    "        # Visualize performance metrics\n",
    "        fig, axes = plt.subplots(1, 3, figsize=(15, 5))\n",
    "        \n",
    "        # Precision\n",
    "        axes[0].bar(performance_df['emotion'], performance_df['precision'], \n",
    "                   color=[EmotionUtils.EMOTION_COLORS.get(e, '#888888') for e in performance_df['emotion']])\n",
    "        axes[0].set_title('Precision by Emotion')\n",
    "        axes[0].set_ylabel('Precision')\n",
    "        axes[0].tick_params(axis='x', rotation=45)\n",
    "        \n",
    "        # Recall\n",
    "        axes[1].bar(performance_df['emotion'], performance_df['recall'],\n",
    "                   color=[EmotionUtils.EMOTION_COLORS.get(e, '#888888') for e in performance_df['emotion']])\n",
    "        axes[1].set_title('Recall by Emotion')\n",
    "        axes[1].set_ylabel('Recall')\n",
    "        axes[1].tick_params(axis='x', rotation=45)\n",
    "        \n",
    "        # F1-Score\n",
    "        axes[2].bar(performance_df['emotion'], performance_df['f1_score'],\n",
    "                   color=[EmotionUtils.EMOTION_COLORS.get(e, '#888888') for e in performance_df['emotion']])\n",
    "        axes[2].set_title('F1-Score by Emotion')\n",
    "        axes[2].set_ylabel('F1-Score')\n",
    "        axes[2].tick_params(axis='x', rotation=45)\n",
    "        \n",
    "        plt.tight_layout()\n",
    "        plt.savefig('../models/image_emotion_model/performance_analysis.png')\n",
    "        plt.show()\n",
    "        \n",
    "        # Best and worst performing emotions\n",
    "        best_f1 = performance_df.loc[performance_df['f1_score'].idxmax()]\n",
    "        worst_f1 = performance_df.loc[performance_df['f1_score'].idxmin()]\n",
    "        \n",
    "        print(f\"\\n🏆 Best performing emotion: {best_f1['emotion']} (F1: {best_f1['f1_score']:.3f})\")\n",
    "        print(f\"🔴 Worst performing emotion: {worst_f1['emotion']} (F1: {worst_f1['f1_score']:.3f})\")\n",
    "        \n",
    "        # Overall metrics\n",
    "        if 'macro avg' in report:\n",
    "            macro_avg = report['macro avg']\n",
    "            print(f\"\\n📊 Overall Performance:\")\n",
    "            print(f\"   Macro Precision: {macro_avg['precision']:.3f}\")\n",
    "            print(f\"   Macro Recall: {macro_avg['recall']:.3f}\")\n",
    "            print(f\"   Macro F1-Score: {macro_avg['f1-score']:.3f}\")\n",
    "    else:\n",
    "        print(\"No emotion-specific performance data available\")\nelse:\n",
    "    print(\"No evaluation results available for analysis\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 8. Save Model Summary"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create model summary\n",
    "model_summary = {\n",
    "    \"model_type\": \"Image Emotion Recognition\",\n",
    "    \"architecture\": \"MobileNetV2 + Classification Head\",\n",
    "    \"training_data\": {\n",
    "        \"total_samples\": len(image_paths) if 'image_paths' in locals() else 0,\n",
    "        \"training_samples\": len(train_loader.dataset) if train_loader else 0,\n",
    "        \"validation_samples\": len(val_loader.dataset) if val_loader else 0,\n",
    "        \"emotions\": list(EmotionUtils.EMOTION_LABELS),\n",
    "        \"image_size\": IMAGE_SIZE\n",
    "

In [ ]:
import sys
import os
sys.path.append('../src')

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from PIL import Image
from sklearn.metrics import classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

from data_loader import DataLoader
from model_trainer import ModelTrainer, ImageEmotionModel
from utils import EmotionUtils, ModelUtils, ImageUtils

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Check GPU availability
device = ModelUtils.get_device()
print(f"Using device: {device}")


## 1. Load and Prepare Image Data

In [ ]:
# Initialize data loader
data_loader = DataLoader(data_dir='../data/')

# Load processed image data
processed_data_path = '../data/processed/image_emotions.csv'

if os.path.exists(processed_data_path):
    df = pd.read_csv(processed_data_path)
    image_paths = df['image_path'].tolist()
    labels = df['emotion'].tolist()
    print(f"Loaded {len(image_paths)} image samples from processed data")
else:
    print("Processed data not found. Loading raw data...")
    # Try to load from FER-2013
    fer_data_path = '../data/fer2013/'
    
    if os.path.exists(fer_data_path):
        image_paths, labels = data_loader.load_fer2013_data(fer_data_path)
    else:
        print("FER-2013 dataset not found. Creating synthetic data for demonstration...")
        # Create synthetic image data for demonstration
        # In real scenario, you would have actual image files
        
        # Create sample images directory
        os.makedirs('../data/sample_images', exist_ok=True)
        
        image_paths = []
        labels = []
        
        # Generate sample grayscale images for each emotion
        for emotion in EmotionUtils.EMOTION_LABELS:
            for i in range(20):  # 20 samples per emotion
                # Create a random 48x48 grayscale image
                img = np.random.randint(0, 256, (48, 48), dtype=np.uint8)
                
                # Add some pattern based on emotion (just for demo)
                if emotion == 'happy':
                    img = np.clip(img + 50, 0, 255)  # Brighter
                elif emotion == 'sad':
                    img = np.clip(img - 50, 0, 255)  # Darker
                
                img_path = f'../data/sample_images/{emotion}_{i:03d}.png'
                cv2.imwrite(img_path, img)
                
                image_paths.append(img_path)
                labels.append(emotion)
        
        print(f"Created {len(image_paths)} synthetic image samples for training")

print(f"\nUnique emotions: {set(labels)}")
print(f"Label distribution:")
from collections import Counter
label_counts = Counter(labels)
for emotion, count in label_counts.items():
    print(f"  {emotion}: {count}")

# Filter out non-existent image paths
valid_paths = []
valid_labels = []
for path, label in zip(image_paths, labels):
    if os.path.exists(path):
        valid_paths.append(path)
        valid_labels.append(label)

image_paths = valid_paths
labels = valid_labels
print(f"\nValid image files: {len(image_paths)}")


In [ ]:
# Visualize sample images
print("Sample Images:")

# Select a few images from each emotion
sample_images = {}
for emotion in EmotionUtils.EMOTION_LABELS:
    emotion_paths = [path for path, label in zip(image_paths, labels) if label == emotion]
    if emotion_paths:
        sample_images[emotion] = emotion_paths[0]  # Take first image of each emotion

if sample_images:
    fig, axes = plt.subplots(2, 4, figsize=(12, 6))
    axes = axes.flatten()
    
    for i, (emotion, img_path) in enumerate(sample_images.items()):
        if i < 8:  # Only show first 8
            try:
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                if img is not None:
                    axes[i].imshow(img, cmap='gray')
                    axes[i].set_title(f'{emotion.capitalize()}')
                    axes[i].axis('off')
                else:
                    axes[i].text(0.5, 0.5, f'{emotion}\n(No image)', 
                               ha='center', va='center', transform=axes[i].transAxes)
                    axes[i].axis('off')
            except Exception as e:
                axes[i].text(0.5, 0.5, f'{emotion}\n(Error loading)', 
                           ha='center', va='center', transform=axes[i].transAxes)
                axes[i].axis('off')
    
    plt.suptitle('Sample Images by Emotion')
    plt.tight_layout()
    plt.show()
else:
    print("No sample images available to display.")


In [ ]:
# Create data loaders
print("Creating data loaders...")

# Configuration
BATCH_SIZE = 32
TEST_SIZE = 0.2
IMAGE_SIZE = (48, 48)

if len(image_paths) > 0:
    train_loader, val_loader, label_encoder = data_loader.create_image_dataloaders(
        image_paths=image_paths,
        labels=labels,
        batch_size=BATCH_SIZE,
        test_size=TEST_SIZE
    )
    
    print(f"Training batches: {len(train_loader)}")
    print(f"Validation batches: {len(val_loader)}")
    print(f"Classes: {label_encoder.classes_}")
else:
    print("❌ No valid image data available for training")
    train_loader = None
    val_loader = None
    label_encoder = None


## 2. Initialize Model and Trainer

In [ ]:
# Initialize model trainer
model_trainer = ModelTrainer(device=device)

# Training configuration
training_config = {
    "model_architecture": "MobileNetV2",
    "num_epochs": 10,  # Reduced for demo
    "learning_rate": 0.001,
    "batch_size": BATCH_SIZE,
    "image_size": IMAGE_SIZE,
    "num_emotions": len(EmotionUtils.EMOTION_LABELS),
    "device": str(device),
    "pretrained": True
}

print("Training Configuration:")
for key, value in training_config.items():
    print(f"  {key}: {value}")

# Save training config
os.makedirs('../models/image_emotion_model', exist_ok=True)
model_trainer.save_training_config(training_config, '../models/image_emotion_model/config.json')


## 3. Train the Model

In [ ]:
# Train the image emotion model
if train_loader is not None and val_loader is not None:
    print("Starting model training...")
    
    model_save_path = '../models/image_emotion_model/model.pth'
    
    history = model_trainer.train_image_model(
        train_loader=train_loader,
        val_loader=val_loader,
        model_save_path=model_save_path,
        num_epochs=training_config["num_epochs"],
        learning_rate=training_config["learning_rate"]
    )
    
    print("\n✅ Training completed!")
else:
    print("❌ Cannot start training - no valid data available")
    # Create a dummy history for demonstration
    history = {
        'train_loss': [0.8, 0.6, 0.4, 0.3, 0.2],
        'val_loss': [0.9, 0.7, 0.5, 0.4, 0.35],
        'train_acc': [60, 70, 80, 85, 90],
        'val_acc': [55, 65, 75, 80, 82]
    }


## 4. Visualize Training Results

In [ ]:
# Plot training history
if 'history' in locals():
    model_trainer.plot_training_history(
        history, 
        save_path='../models/image_emotion_model/training_history.png'
    )
else:
    print("No training history available to plot")


## 5. Evaluate the Model

In [ ]:
# Load and evaluate the trained model
model_save_path = '../models/image_emotion_model/model.pth'

if os.path.exists(model_save_path) and val_loader is not None:
    # Load the best trained model
    best_model = model_trainer.load_trained_model(model_save_path, "image")
    
    if best_model is not None:
        print("Model loaded successfully!")
        
        # Evaluate on validation set
        print("\nEvaluating model...")
        evaluation_results = model_trainer.evaluate_model(
            model=best_model,
            test_loader=val_loader,
            model_type="image"
        )
        
        print(f"\nValidation Accuracy: {evaluation_results['accuracy']:.4f}")
        
        # Print classification report
        print("\nClassification Report:")
        report = evaluation_results['classification_report']
        for emotion in EmotionUtils.EMOTION_LABELS:
            if emotion in report:
                metrics = report[emotion]
                print(f"  {emotion}:")
                print(f"    Precision: {metrics['precision']:.3f}")
                print(f"    Recall: {metrics['recall']:.3f}")
                print(f"    F1-score: {metrics['f1-score']:.3f}")
        
        # Plot confusion matrix
        model_trainer.plot_confusion_matrix(
            evaluation_results['confusion_matrix'],
            save_path='../models/image_emotion_model/confusion_matrix.png'
        )
    else:
        print("❌ Failed to load trained model")
else:
    print("❌ No trained model available or no validation data")
    best_model = None


## 6. Test Model with Sample Images

In [ ]:
# Test the model with sample images
if best_model is not None and len(image_paths) > 0:
    print("Testing model with sample images:\n")
    
    # Select a few test images
    test_indices = np.random.choice(len(image_paths), min(5, len(image_paths)), replace=False)
    
    fig, axes = plt.subplots(1, len(test_indices), figsize=(15, 3))
    if len(test_indices) == 1:
        axes = [axes]
    
    for i, idx in enumerate(test_indices):
        img_path = image_paths[idx]
        true_label = labels[idx]
        
        # Load and preprocess image
        try:
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is not None:
                # Predict emotions
                emotion_scores = model_trainer.predict_single_image(
                    model=best_model,
                    image=img
                )
                
                # Get predicted emotion
                predicted_emotion = max(emotion_scores.items(), key=lambda x: x[1])[0]
                confidence = emotion_scores[predicted_emotion]
                
                # Display image
                axes[i].imshow(img, cmap='gray')
                axes[i].set_title(f'True: {true_label}\nPred: {predicted_emotion}\n({confidence:.2f})')
                axes[i].axis('off')
                
                print(f"Image {i+1}: {img_path}")
                print(f"  True emotion: {true_label}")
                print(f"  Predicted emotion: {predicted_emotion} (confidence: {confidence:.3f})")
                
                # Show top 3 predictions
                sorted_emotions = sorted(emotion_scores.items(), key=lambda x: x[1], reverse=True)[:3]
                print("  Top predictions:")
                for emotion, score in sorted_emotions:
                    emoji = EmotionUtils.get_emotion_emoji(emotion)
                    print(f"    {emoji} {emotion}: {score:.3f}")
                print()
            else:
                axes[i].text(0.5, 0.5, 'Failed to\nload image', 
                           ha='center', va='center', transform=axes[i].transAxes)
                axes[i].axis('off')
        except Exception as e:
            print(f"Error processing image {img_path}: {e}")
            axes[i].text(0.5, 0.5, f'Error:\n{str(e)[:20]}', 
                       ha='center', va='center', transform=axes[i].transAxes)
            axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()
else:
    print("❌ Model not available for testing or no images available")


## 7. Model Performance Analysis

In [ ]:
# Analyze model performance by emotion
if 'evaluation_results' in locals():
    print("Model Performance Analysis:\n")
    
    # Extract metrics for each emotion
    emotions_data = []
    report = evaluation_results['classification_report']
    
    for emotion in EmotionUtils.EMOTION_LABELS:
        if emotion in report:
            emotions_data.append({
                'emotion': emotion,
                'precision': report[emotion]['precision'],
                'recall': report[emotion]['recall'],
                'f1_score': report[emotion]['f1-score'],
                'support': report[emotion]['support']
            })
    
    # Create DataFrame for easier analysis
    if emotions_data:
        performance_df = pd.DataFrame(emotions_data)
        
        print("Per-Emotion Performance:")
        print(performance_df.round(3))
        
        # Visualize performance metrics
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))
        
        # Precision
        axes[0].bar(performance_df['emotion'], performance_df['precision'], 
                   color=[EmotionUtils.EMOTION_COLORS.get(e, '#888888') for e in performance_df['emotion']])
        axes[0].set_title('Precision by Emotion')
        axes[0].set_ylabel('Precision')
        axes[0].tick_params(axis='x', rotation=45)
        
        # Recall
        axes[1].bar(performance_df['emotion'], performance_df['recall'],
                   color=[EmotionUtils.EMOTION_COLORS.get(e, '#888888') for e in performance_df['emotion']])
        axes[1].set_title('Recall by Emotion')
        axes[1].set_ylabel('Recall')
        axes[1].tick_params(axis='x', rotation=45)
        
        # F1-Score
        axes[2].bar(performance_df['emotion'], performance_df['f1_score'],
                   color=[EmotionUtils.EMOTION_COLORS.get(e, '#888888') for e in performance_df['emotion']])
        axes[2].set_title('F1-Score by Emotion')
        axes[2].set_ylabel('F1-Score')
        axes[2].tick_params(axis='x', rotation=45)
        
        plt.tight_layout()
        plt.savefig('../models/image_emotion_model/performance_analysis.png')
        plt.show()
        
        # Best and worst performing emotions
        best_f1 = performance_df.loc[performance_df['f1_score'].idxmax()]
        worst_f1 = performance_df.loc[performance_df['f1_score'].idxmin()]
        
        print(f"\n🏆 Best performing emotion: {best_f1['emotion']} (F1: {best_f1['f1_score']:.3f})")
        print(f"🔴 Worst performing emotion: {worst_f1['emotion']} (F1: {worst_f1['f1_score']:.3f})")
        
        # Overall metrics
        if 'macro avg' in report:
            macro_avg = report['macro avg']
            print(f"\n📊 Overall Performance:")
            print(f"   Macro Precision: {macro_avg['precision']:.3f}")
            print(f"   Macro Recall: {macro_avg['recall']:.3f}")
            print(f"   Macro F1-Score: {macro_avg['f1-score']:.3f}")
    else:
        print("No emotion-specific performance data available")
else:
    print("No evaluation results available for analysis")


## 8. Save Model Summary

In [ ]:
# Create model summary
model_summary = {
    "model_type": "Image Emotion Recognition",
    "architecture": "MobileNetV2 + Classification Head",
    "training_data": {
        "total_samples": len(image_paths) if 'image_paths' in locals() else 0,
        "training_samples": len(train_loader.dataset) if train_loader else 0,
        "validation_samples": len(val_loader.dataset) if val_loader else 0,
        "emotions": list(EmotionUtils.EMOTION_LABELS),
        "image_size": IMAGE_SIZE
    }
}
